# A. Importing Library

In [1]:
from sklearn import metrics
from sklearn import decomposition
from sklearn import manifold
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.svm import SVR
from sklearn.neural_network import MLPClassifier,MLPRegressor
from sklearn.model_selection import GridSearchCV

import tensorflow as tf
from tensorflow import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import np_utils
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D

from numpy.random import randn
from numpy.random import seed
from scipy.stats import pearsonr
from matplotlib.pyplot import figure
from sklearn.metrics import accuracy_score, precision_score, recall_score

import joblib
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

# B. Defining functions

In [2]:
# nucleotide to unit vector
def one_hot(base):
  return np.array({'a':[1,0,0,0], 
                   't':[0,1,0,0], 
                   'g':[0,0,1,0], 
                   'c':[0,0,0,1]}[base.lower()])

# replace nucleotide for unit vector using one_hon()
def Hot(seq): 
  seq_one_hot = np.zeros((len(seq)*4, )) # zero 200,1 matrix 
  for i in range(len(seq)): seq_one_hot[4*i : 4*i+4] = one_hot(seq[i])
  return seq_one_hot

# normalize data (-1 to +1)
def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

# print results function
def print_results(results):
    print('Best Params: {}\n'.format(results.best_params_))
    
    means = results.cv_results_['mean_test_score']
    stds = results.cv_results_['std_test_score']
    for mean, std, params in zip(means,stds, results.cv_results_['params']):
        print('{} (+/-{}) for {}'.format(round(mean,3),round(std*2,3),params))

        
def clean_data(d1):
    # change column name
    d2 = d1.rename(columns={d1.columns[0]:'index',d1.columns[1]:'seq100',d1.columns[2]:'c26',d1.columns[3]:'c29', d1.columns[4]:'c31',d1.columns[5]:'c0'}) 


    # remove overhanger
    seq =[]
    for i in range(len(d2['seq100'])):
        seq.append(d2['seq100'][i][25:75])
    d3 = pd.DataFrame(seq,columns=['seq50'])
    d4 = pd.concat([d2,d3],axis=1)
    df = d4.drop(['index','seq100'],axis=1)

    #take a random sample of 10000
    df = df.sample(n=len(df))
    return  df

def encode_dna_seq(df):
    # extract column of sequence
    SeqList = df['seq50'].tolist()

    # extract c0 values
    dy = np.around(df['c0'].values,3) 

    # encode dna sequence
    d = [] 
    for i in range(len(SeqList)): 
        d.append(Hot(SeqList[i])) 
    d = np.array(d)
    dx = pd.DataFrame(d)
    input = dx.values
    return input,dy

def split_data_tvt(dx,dy):
    x_train, x_test, y_train, y_test  = train_test_split(dx,dy,shuffle=True, test_size=0.40, random_state=1234)
    x_val, x_test, y_val, y_test  = train_test_split(x_test,y_test,shuffle=True, test_size=0.50, random_state=1234)
    return x_train, y_train, x_val, y_val, x_test, y_test

# C. Importing data

In [3]:
d = pd.read_csv('C:/Users/bciez/Documents/Basilio/jhu/ha-lab/dna_flexibility/thesis_01_18_2022/data/dataset3.txt',delimiter='\t')
print('ready')

ready


# D. Cleaning data

In [4]:
df = clean_data(d)
df.head(3)

,c26,c29,c31,c0,Amplitude,Phase,seq50
5146,-0.272351,0.190476,-0.226578,-0.282006,0.401526,0.024049,GGGACCCGAGCGGACGCGCAGCTACCACAGTTGCAGCTAACATATC...
2435,-0.432030,-0.326253,-0.079366,-0.280802,-0.169098,1.106908,ATCTCATGAAATTAGTTGTAGTATAGTAAGGCTATCCTTTCCACGC...
11488,-0.279292,-0.308483,-0.255158,-0.266567,-0.040582,0.318931,TGGCGGGGGCGGCCTCCACACTTACTCTTTAGTAGGATGTCTATTT...


# E. Encoding DNA sequence

In [5]:
dx,dy = encode_dna_seq(df)
print('ready')

ready


# F. Split data in training and testing set

In [6]:
x_train, y_train, x_val, y_val, x_test, y_test = split_data_tvt(dx,dy)
print('ready')

ready


# G. Support Vector Machine

In [7]:
svc = SVR()
parameters = {
    'kernel': ['linear','rbf'],
    'C': [0.1,1,10]
}

cv = GridSearchCV(svc, parameters, cv = 5)
cv.fit(x_train, y_train)

print_results(cv)

Best Params: {'C': 1, 'kernel': 'rbf'}

-0.033 (+/-0.013) for {'C': 0.1, 'kernel': 'linear'}
0.009 (+/-0.01) for {'C': 0.1, 'kernel': 'rbf'}
-0.033 (+/-0.013) for {'C': 1, 'kernel': 'linear'}
0.078 (+/-0.008) for {'C': 1, 'kernel': 'rbf'}
-0.033 (+/-0.013) for {'C': 10, 'kernel': 'linear'}
0.075 (+/-0.008) for {'C': 10, 'kernel': 'rbf'}


# H. Saving model

In [8]:
joblib.dump(cv.best_estimator_,'svm_model.pkl')

['svm_model.pkl']

# I. Best Model

In [9]:
cv.best_estimator_

SVR(C=1, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)